## **04 텍스트 분류 실습 - 20 뉴스그룹 분류**

**사이킷런이 내부에 가지고 있는 예제 데이터인 20 뉴스 그룹 데이터 세트를 이용해 텍스트 분류를 적용해본다. 텍스트 분류는 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것이다.**

![test](./img/실습1.png)

**사이킷런은 fetch_20newsgroups() API를 이용해 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공한다.**

**텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 된다. 그리고 이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등이다. 이 중 로지스틱 회귀를 이용해 분류를 수행해 본다. 텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용한다. 그리고 그 이후에 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가한다.**

**이번 절에서는 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용해 예측 성능을 비교하고, 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝, 그리고 사이킷런의 Pipeline 객체를 통해 피처 벡터화 파라마터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행하는 방법을 소개한다.**

### **텍스트 정규화**

In [3]:
# fetch_20newsgroups()는 인터넷에서 로컬 컴퓨터로 데이터를 먼저 내려받은 후에 메모리로 데이터를 로딩한다.
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [4]:
# fetch_20newsgroups()는 사이킷런의 다른 데이터 세트 예제와 같이 파이썬 딕셔너리와 유사한 Bunch 객체를 반환한다. 어떤한 Key 값을 가지고 있는지 확인
# fetch_20newsgroups() API 역시 load_xxx() API와 유사한 Key 값을 가지고 있다. 
# filenames라는 key 이름이 눈에 띄는데, 이는 fetch_20newsgroups() API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭한다.
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [7]:
# Target 클래스가 어떻게 구성돼 있는지 확인
# Target 클래스의 값은 0부터 19까지 20개로 구성돼 있다.
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
# 개별 데이터가 어떻게 구성돼 있는지 데잍를 한 개만 추출해 값을 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

**텍스트 데이터를 확인해 보면 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있다. 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거한다. 왜냐하면 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들을 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경구가 많기 때문이다.**

> **이 피처들을 포함하게 되면 왠만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타낸다. 따라서 이들 헤더와 푸터 정보를 포함하는 것은 이 장에서 수행하려는 텍스트 분석의 의도를 벗어나기에 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류할 것이다.**

In [13]:
# remove 파라미터를 이용하면 뉴스그룹 기사의 헤더(header), 푸터(footer) 등을 제거할 수 있다.
# 또한 fetch_20newsgroups()는 subset 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있다.
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'),random_state=156)
X_test= test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### **피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**

**CounterVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화를 한다. 테스트 데이터 역시 피처 벡터화를 수행하는데, 한 가지 반드시 유의해야 할 점이 있다. 바로 테스트 데이터에서 CounterVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CounterVectorizer 객체를 이용해 테스트 데이터를 변환(transform)해야 한다는 것이다. 그래야만 학습 시 설정된 CounterVectorizer의 피처 개수와 테스트 데이터를 CounterVectorizer로 변환할 피처 개수가 같아진다.**

**테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CounterVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환한다.**

**테스트 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안 된다는 점도 유의해야한다. CounterVectorizer.fit_transform(테스트 데이터)을 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 CounterVectorizer가 fit()을 수행하고 transform()하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라진다.**

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()

cnt_vect.fit(X_train,y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

# 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11314개의 문서에서 피처, 즉 단어가 101631개로 만들어졌다.
print('학습 데이터 텍스트의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [18]:
# 이렇게 피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류를 예측한다.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)

# Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀 정확도는 약 0.617이다.
print('CountVectorizer Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CountVectorizer Logistic Regression의 예측 정확도는 0.617


In [21]:
# 이번에는 Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델을 수행한다.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


TF-IDF Logistic Regression의 예측 정확도는 0.678


**TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공한다. 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.**

**텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것이다.**

> **텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있다.**

In [22]:
# 앞의 TF-IDF 벡터화는 기본 파라미터만 적용했지만, 이번에는 좀 더 다양한 파라미터를 적용해본다.
# TfidfVectorizer 클래스의 스톱 워드를 기존의 'None'에서 'english'로 변경하고, 
# ngram_range는 기존 (1,1)에서 (1,2)로, max_df=300으로 변경한 뒤 다시 예측 성능을 측정해본다.

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
# tfidf_vect.vocabulary_ # 전처리 결과
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.690


In [26]:
# 이번에는 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼파라미터 최적화를 수행한다.
# 로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C 값을 찾은 뒤 이 C 값을 찾은 뒤 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능을 평가

from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. cv는 3 폴드 세트로 설정
params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.7min finished


Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


**로지스틱 회귀의 C가 10일 때 GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냈으며, 이를 테스트 데이터 세트에 적용해 약 0.703으로 이전보다 약간 향상된 성능 수치가 됐다.**

### **사이컷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합**

**사이킷런의 Pipeline 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있다. 일반적으로 머신러닝에서 Pipeline이란 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 마치 수도관(Pipe)에서 물이 흐르듯 한꺼번에 스트림 기반으로 처리한다는 의미이다.**

**이렇게 Pipeline을 이용하면 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있다. 또한 대용량의 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 수행 시간을 절약할 수 있다.** 

**일반적으로 사이킷런 파이프라인은 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있다.**

> **예를 들어 스케일링 또는 벡터 정규화, PCA 등의 변환 작업과 분류, 회귀 등의 Estimator를 한 번에 결합하는 것이다.**

In [28]:
# 다음은 위에서 텍스트 분류 예제 코드를 Pipeline을 이용해 다시 작성한 코드이다. Pipeline 객체는 다음과 같이 선언
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                     ('lr_clf', LogisticRegression(random_state=156))])

**이것은 TfidfVectorizer 객체를 tfidf_vect라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성한 뒤 이 두개의 객체를 파이프라인으로 연결하는 Pipeline을 생성한다는 의미이다.**

**또한 다음 코드를 보면 기존 TfidfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit()과 transform() 수행을 통한 피처 벡터화와 LogisticRegressor의 fit()과 predict() 수행을 통한 머신러닝 모델의 학습과 예측이 Pipeline의 fit()과 predict() 통일돼 수행됨을 알 수 있다. 이렇게 Pipelione 방식을 적용하면 머신러닝 코드를 더 직관적이고 쉽게 작성할 수 있다.**

In [29]:
# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

In [30]:
# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegressor의 fit(), predict()가 필요없다.
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.704


**사이킷런은 GridSearchCV 클래스의 생성 파라미터로 Pipeline을 입력해 Pipeline 기반에서도 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행할 수 있게 지원한다. 이렇게 하면 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터를 모두 한 번에 GridSearchCV를 이용해 최적화할 수 있다.**

**다음 예제는 GridSearchCV에 Pipeline을 입력하면서 TfidfVectorizer의 파라미터와 LogisticRegression의 하이퍼 파라미터를 함께 최적화한다. 예제 코드를 잠깐 살펴보면 GridSearchCV에 Estimator가 아닌 Pipeline을 입력할 경우에는 param_grid의 입력 값 설정이 기존과 약간 다르다.**

**딕셔너리 형태의 Key와 Value 값을 가지며, Value를 리스트 형태로 입력하는 것은 동일하다. 다만 Key 값을 살펴보면 tfidf_vect__ngram_range와 같이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공된다.**

**Pipeline을 GridSearchCV에 인자로 입력하면 GridSearchCV는 Pipeline을 구성하는 피처 벡터화 객체의 파라미터와 Estimator 객체의 하이퍼 파라미터를 각각 구별할 수 있어야 하는데, 이때 개별 객체 명과 파라미터명/하이퍼 파라미터명을 결합해 Key 값으로 할당하는 것이다.**

> **가령 TfdifVectorizer 객체 변수인 tfdif_vect의 n_gram_range 파라미터 값을 변화시키면서 최적화하기를 원한다면 객체 변수명인 tfdif_vect에 언더바 2개를 연달아 붙인 뒤 파라미터명인 n_gram_range를 결합해 'tfidf_vect__ngram_range'를 Key 값으로 할당하는 것이다.**

**Pipeline + GridSearchCV를 적용할 때 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점이다. 피처 벡터화에 사용되는 파라미터와 GridSearchCV 하이퍼 파라미터를 합치면 최적화를 위한 너무 많은 경우의 수가 발생하기 쉽다.**

In [32]:
# 다음 예제의 경우 Pipeline + GridSearchCV 기반으로 하이퍼 파라미터 튜닝을 적용해 27개의 파라미터 경우의 수 X 3개의 cv로 총 81번의 학습과 검증을 수행
# 랩탑 PC에서 약 24분이 소모
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정
params = {
    'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf_vect__max_df':[100,300,700],
    'lr_clf__C':[1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.755524129397207
Pipeline을 통한 Logistic Regression의 예측 정확도는 0.702


**TfidfVectorizer 객체의 max_df 파라미터가 700, ngram_range 파라미터가 (1,2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C 하이퍼 파라미터에 10을 적용해 예측 분류를 수행할 때 가장 좋은 검증 세트 성능 수치가 도출했다. 아쉽게도 이렇게 최적화한 파라미터를 기반으로 테스트 데이터 세트에 대해 예측했을 때의 정확도는 약 0.702로 크게 개선은 되자 않았다.**

**로지스틱 회귀 외에 서포트 벡터머신과 나이브 베이즈 알고리즘도 희소 행렬 기반의 텍스트 분류에 자주 사용되는 머신러닝 알고리즘이다. 이들을 이용한 모델도 만들어 볼 것을 권장한다.**